## Text-from-Embedding (Russian SBERT -> Russian BART)

Восстанавливаем (или перефразируем) текст на русском языке, имея на входе **ровно один 1024-мерный SBERT-вектор** (например, от `sberbank-ai/sbert_large_nlu_ru`).
Используется замороженный русскоязычный BART-декодер (`antoinelouis/bart-base-russian`), обучается только небольшой MLP-проектор.

In [1]:
# @markdown Выполните эту ячейку для установки зависимостей и клонирования репозитория.

GIT_REPO_URL = "https://github.com/maxxxsudb/Text-from-Embedding.git" # @param {type:"string"}

!git clone $GIT_REPO_URL text-from-embedding
%cd text-from-embedding
!bash setup.sh
!pip freeze | grep -E "torch|transformers|sentence-transformers|sacrebleu"

Cloning into 'text-from-embedding'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 47 (delta 19), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 22.46 KiB | 11.23 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/text-from-embedding
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s

In [2]:
# @markdown Создадим небольшие `train.jsonl` и `val.jsonl` для демонстрации.
# @markdown В реальном проекте используйте `prepare_embeddings_script.py` на вашем большом корпусе.

import json
from sentence_transformers import SentenceTransformer
import os

# Параметры
SBERT_MODEL_NAME = "sberbank-ai/sbert_large_nlu_ru" # 1024-dim
SBERT_DIM = 1024 # Должно соответствовать модели SBERT

# Создаем директорию для данных, если ее нет
os.makedirs("data", exist_ok=True)

# Пример текстов
train_texts = [
    "Это первое предложение для обучающей выборки.",
    "Модель должна научиться генерировать похожий текст.",
    "Пример качественного описания продукта для интернет-магазина.",
    "Новости технологий и искусственного интеллекта."
]
val_texts = [
    "Валидационный пример для проверки модели.",
    "Короткое предложение для теста."
]

def create_jsonl_data(texts, output_path, sbert_model):
    print(f"Generating embeddings for {output_path} using {SBERT_MODEL_NAME}...")
    with open(output_path, "w", encoding="utf-8") as f_out:
        # Кодируем все тексты сразу для эффективности
        embeddings = sbert_model.encode(texts, convert_to_tensor=False, normalize_embeddings=False)
        for text, vec in zip(texts, embeddings):
            if vec.shape[0] != SBERT_DIM:
                raise ValueError(f"SBERT model {SBERT_MODEL_NAME} produced embeddings of dim {vec.shape[0]}, expected {SBERT_DIM}")
            record = {"embedding": vec.tolist(), "text": text}
            f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
    print(f"Saved {len(texts)} records to {output_path}")

# Загрузка SBERT модели
print(f"Loading SBERT model: {SBERT_MODEL_NAME}...")
sbert = SentenceTransformer(SBERT_MODEL_NAME)
print("SBERT model loaded.")

# Создание файлов данных
create_jsonl_data(train_texts, "data/train_demo.jsonl", sbert)
create_jsonl_data(val_texts, "data/val_demo.jsonl", sbert)

print("\nСодержимое data/train_demo.jsonl (первая строка):")
!head -n 1 data/train_demo.jsonl

Loading SBERT model: sberbank-ai/sbert_large_nlu_ru...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

SBERT model loaded.
Generating embeddings for data/train_demo.jsonl using sberbank-ai/sbert_large_nlu_ru...
Saved 4 records to data/train_demo.jsonl
Generating embeddings for data/val_demo.jsonl using sberbank-ai/sbert_large_nlu_ru...
Saved 2 records to data/val_demo.jsonl

Содержимое data/train_demo.jsonl (первая строка):
{"embedding": [0.056174878031015396, -0.020228281617164612, 0.03529005125164986, 0.012322106398642063, 0.009066455066204071, -0.0218281839042902, 0.0010669180192053318, 0.005263828672468662, 0.017435627058148384, -0.024311872199177742, 0.008098071441054344, 0.017549259588122368, 0.04357627034187317, -0.019435616210103035, -0.017171615734696388, -0.02676442638039589, 0.02930869162082672, -0.03241724893450737, -0.018186835572123528, 0.03477214276790619, -0.03773966804146767, -0.0395830012857914, 0.014302518218755722, 0.011393018066883087, -0.037150800228118896, -0.01683039590716362, 0.004618261940777302, -0.009967023506760597, -0.02554137632250786, -0.01369075942784547

In [3]:
# @markdown Запускаем обучение. Для демо используются минимальные параметры.
# @markdown В реальном проекте увеличьте `--epochs`, `--bs`, и настройте `--lr`, `--proj_bottleneck_dim` и др.

# Параметры обучения
BART_MODEL = "antoinelouis/bart-base-russian" # @param {type:"string"}
SBERT_DIM_TRAIN = 1024 # Убедитесь, что это значение соответствует SBERT_DIM из предыдущей ячейки и модели SBERT
SAVE_DIR = "checkpoints/demo_run_russian" # @param {type:"string"}
EPOCHS = 2 # @param {type:"integer"}
BATCH_SIZE = 2 # @param {type:"integer"}
LEARNING_RATE = 5e-4 # @param {type:"number"}
PROJECTOR_K = 3 # @param {type:"integer"}
PROJECTOR_BOTTLENECK_DIM = 1024 # @param {type:"integer"} # 0 или отрицательное для однослойного; SBERT_DIM для прямого 2-слойного без сужения

!python -m src.train \
  --train_jsonl data/train_demo.jsonl \
  --val_jsonl   data/val_demo.jsonl \
  --save_dir    $SAVE_DIR \
  --bart_model  $BART_MODEL \
  --sbert_dim   $SBERT_DIM_TRAIN \
  --epochs      $EPOCHS \
  --bs          $BATCH_SIZE \
  --lr          $LEARNING_RATE \
  --warmup_steps 5 \
  --k           $PROJECTOR_K \
  --proj_bottleneck_dim $PROJECTOR_BOTTLENECK_DIM \
  --max_len     64 \
  --max_new_tokens_val 32 \
  --num_beams_val 2 \
  --num_workers 2 \
  --label_smoothing 0.0 # Для демо можно отключить, чтобы быстрее сходилось на малых данных

2025-05-06 19:11:37.160577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746558697.185702    1482 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746558697.192785    1482 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/antoineloui

In [ ]:
# @markdown Используем обученную модель для генерации текста из SBERT-вектора.
# @markdown Возьмем вектор из нашего валидационного датасета.

import numpy as np
import json
import os
import torch

# Переменная SAVE_DIR должна быть доступна из предыдущей ячейки
# Если ячейки выполняются не по порядку, определите SAVE_DIR здесь:
# SAVE_DIR = "checkpoints/demo_run_russian"

# Загрузим первый вектор из валидационного набора
val_vector_np = None
with open("data/val_demo.jsonl", "r", encoding="utf-8") as f:
    first_line = f.readline()
    if first_line:
        record = json.loads(first_line)
        val_vector_np = np.array(record["embedding"])
        original_text = record["text"]
        print(f"Оригинальный текст для вектора: {original_text}")

if val_vector_np is not None:
    # Сохраняем вектор в .npy для передачи в скрипт generate.py
    np.save("temp_val_vec.npy", val_vector_np)
    print(f"Вектор сохранен в temp_val_vec.npy, форма: {val_vector_np.shape}")

    # Путь к лучшему чекпоинту
    CKPT_PATH = f"{SAVE_DIR}/best_bleu_model.pt"

    if not os.path.exists(CKPT_PATH) and os.path.exists(f"{SAVE_DIR}/last_checkpoint.pt"):
        print(f"Файл {CKPT_PATH} не найден. Попытка использовать model_state_dict из last_checkpoint.pt...")
        checkpoint = torch.load(f"{SAVE_DIR}/last_checkpoint.pt", map_location="cpu")
        if 'model_state_dict' in checkpoint:
            torch.save(checkpoint['model_state_dict'], CKPT_PATH)
            print(f"model_state_dict из last_checkpoint.pt сохранен как {CKPT_PATH}")
        else:
            print("Ошибка: не удалось извлечь model_state_dict из last_checkpoint.pt. Проверьте структуру чекпоинта.")
            CKPT_PATH = None # Не используем чекпоинт, если он не корректен
    elif not os.path.exists(CKPT_PATH):
        print(f"Ошибка: Файл чекпоинта {CKPT_PATH} не найден, и last_checkpoint.pt также отсутствует или не содержит model_state_dict.")
        CKPT_PATH = None

    if CKPT_PATH:
        # Параметры генерации
        NUM_BEAMS = 3 # @param {type:"integer"}
        MAX_NEW_TOKENS = 50 # @param {type:"integer"}
        MIN_NEW_TOKENS = 5 # @param {type:"integer"}

        !python -m src.generate \
          --ckpt $CKPT_PATH \
          --vec  "temp_val_vec.npy" \
          --num_beams $NUM_BEAMS \
          --max_new_tokens $MAX_NEW_TOKENS \
          --min_new_tokens $MIN_NEW_TOKENS
          # Параметры архитектуры модели (bart_model, sbert_dim, k, proj_bottleneck_dim)
          # будут автоматически загружены из train_args.json, который был сохранен во время обучения
          # в той же директории, что и чекпоинт.
    else:
        print("Инференс не может быть выполнен без корректного файла чекпоинта.")
else:
    print("Не удалось загрузить вектор из data/val_demo.jsonl")

In [ ]:
# @markdown Эта ячейка выведет содержимое файла `train_args.json`,
# @markdown который сохраняется во время обучения и используется при инференсе
# @markdown для восстановления архитектуры модели.

import json
import os

# Переменная SAVE_DIR должна быть доступна из предыдущей ячейки
# Если ячейки выполняются не по порядку, определите SAVE_DIR здесь:
# SAVE_DIR = "checkpoints/demo_run_russian"

TRAIN_ARGS_PATH = f"{SAVE_DIR}/train_args.json"

if os.path.exists(TRAIN_ARGS_PATH):
  with open(TRAIN_ARGS_PATH, "r") as f:
    train_args_content = json.load(f)
    print(json.dumps(train_args_content, indent=2, ensure_ascii=False))
else:
  print(f"Файл {TRAIN_ARGS_PATH} не найден.")